## CIP-50 Leverage-based Pool Ranking Analysis
CCGarant, 7/27/22, @TheStophe

Supporting work for the CIP-50 leveraged-base pool ranking system, comparing Daedalus ranking to CIP-50 Ranking.




### Ranking Equation

The recommended ranking equation starts with a highest score of 10. The pools are down-ranked solely based on leverage and fees factors.

    //equation
    ranking_score = 10 - leverage_factor_term - fee_factor_term

    //variables
    leverage_factor_term = 10 * max{ (pool_leverage/L)^A, (pool_stake/saturation_stake)^B }
    fee_factor_term = C * pool_fee_margin

    //parameters
    - A is 2, has range (0,10), can be tweaked
    - B is 2, has range (0,10), can be tweaked
    - pool_leverage = delegation / pledge
    - pool_stake = pledge + delegation
    - saturation_stake = pool_size soft-cap (e.g., 68M Ada based on k-parameter)
    - C is 50, has range (0-100) ish to be harsh, can be tweaked
    - pool_fee_margin is in range (0-100)% (fixed fee + margin combined)

To evaluate rank using the current reward scheme:

    //current reward scheme only
    ranking_score = 10 - leverage_factor_term - fee_factor_term - fixed_fee_factor

    //variables
    //if fee = minFee, term drops out
    //if fee > minFee, term nonzero but loses relevance w/stake
    //fixed fee maters less to rewards as stake grows, so to here
    fixed_fee_factor = D * (fee-minFee) / stake 

    //parameters
    - D is 50, has range (0,100), can be tweaked
    - fee cannot be less than minFee

### Ranking System

The pool ranking system will be a descending sorted list of ranking scores, with the highest score at the top:

    ranking = sort(ranking_score, order=descending)

The ranking score is intentionally simple and familiar. Pools will be ranked out of a score of 0-10, and "Graded" for ease per below. There can be many pools having the same ranking score.  All wallets should make transparent the knock-down factors that drove the pool's score.

| Grade | Score |
| --- | --- |
| A | 10-9 |
| B | 8-7 |
| C | 6-5 |
| D | 4-3 |
| F | 2-0 |

In [21]:
import numpy as np
import pandas as pd
import bokeh as bk
import matplotlib.pyplot as plt

## Other Ranking Systems

- Adapools.org: [https://github.com/cexplorer/cexplorer.io/blob/main/doc/ranking.md](https://github.com/cexplorer/cexplorer.io/blob/main/doc/ranking.md)
- Eternl:
- Daedalus: 